# Construction of Reduction, Interpolation and Hodge operators on a periodic 1D grid of $[0,1]$

Consider a periodic 1D interval $[0,1]$. The primal grid consists of the nodes of a uniform grid of $N$ cells and the dual grid consists of the midpoints of the cells.
We aim at constructing interpolation operators for point values and cell integrals on the primal and dual grids, with the help of centred Lagrange interpolation. 

This exercise can be done with the class *lagrange*  from *scipy.interpolate*, in particular the *deriv* and *integ* methods.

In all cases we test our schemes by interpolating the function
$\psi: x \mapsto \sin{\left(4\pi x\right)}$.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.linalg    import circulant

from ex2_common import staggered_grid, print_error_analysis
from lagrange_interpolation import Basis_V0, Interpolator_from_C0_to_V0, Interpolant_V0
from lagrange_interpolation import Basis_V1, Interpolator_from_C1_to_V1, Interpolant_V1

In [ ]:
# Domain (periodic)
xmin = 0.0
xmax = 1.0

# Number of (uniform) cells in domain
N = 10

# Cell size, primal grid, and dual grid
dx, xp, xd = staggered_grid(xmin, xmax, N)

# Analytical function for tests
f = lambda x: np.sin(4 * np.pi * x)

# Very fine grid for plots
x = np.linspace(xmin, xmax, 1000, endpoint=False)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.plot(xp, np.zeros_like(xp), 'k.', ms=10, label='Primal grid')
ax.plot(x, f(x), label=r'$\psi\;\; \in \Lambda_0$')
ax.plot(xp, f(xp), 'ko', mfc='None', ms=8, mew=1.5, label=r'$\psi^0 \in C_0$')
ax.legend(fontsize=14)

for xpi in xp:
    ax.plot([xpi, xpi], [0, f(xpi)], 'k--')

ax.grid()
ax.set_title(r'Reduction on primal grid, $\mathcal{R}_0: \Lambda_0 \to C_0$')
ax.set_xlabel('x', size=14)
fig.canvas.draw()

## Interpolation of point values on the primal grid ($I_0: C_{0} \to V_{0}$)

Our goal is to reconstruct a piecewise polynomial function $\psi_h(x) \in V_0$ on the primal grid, given the point values $\psi^0 \in C_0$. Thanks to the uniform grid and periodic boundary conditions, the interpolation operator $\mathcal{I}_0$ is translation invariant and can be defined on a generic interval $[x_i, x_{i+1}]$.

For simplicity we introduce the normalized coordinate $\xi := \frac{x - x_i}{\Delta x}$ and focus on Lagrange interpolation of odd degree $d = 2p+1$, with $p\in\mathbb{N}$, on the $d+1$ grid points $\xi_j \in \left\{ -p, \dots, p+1 \right\}$. The interpolating polynomial can be written in terms of the canonical basis as

$$
P^d_i(\xi) := \sum_{j=-p}^{p+1} \psi^0_{i+j}~\ell^d_j(\xi)
\qquad
\text{with}
\qquad
\ell^d_j(\xi) := \prod_{\substack{k=-p \\ k\neq j}}^{p+1} \frac{\xi - k}{j - k}.
$$

We recall that the Lagrange basis functions satisfy the properties

$$
\ell^d_j(\xi_k) = \delta_{jk},
\qquad
\text{and}
\quad
\sum_{j=-p}^{p+1} \ell^d_j(\xi) = 1
\quad
\forall \xi.
$$

Finally, the interpolation operator is

$$
\mathcal{I}_0: \psi^0 \mapsto \psi_h,
\qquad
\text{with}
\quad
\psi_h: x \mapsto P^d_i\left(\frac{x-x_i}{\Delta x}\right)
\qquad
\text{and}
\quad
i = \left\lfloor {\frac{x - x_0}{\Delta x}} \right\rfloor.
$$

### Degree 1

We use the points $\xi_0 = 0$ and $\xi_1 = 1$ to define the Lagrange basis functions

\begin{align}
\ell^1_0(\xi) &= \frac{\xi - 1}{0 - 1} = 1 - \xi \\
\ell^1_1(\xi) &= \frac{\xi - 0}{1 - 0} = \xi
\end{align}

It is easy to verify that $\sum_j \ell^1_j \equiv 1$.

In [ ]:
l1 = Basis_V0(degree=1)

In [ ]:
xi = np.linspace(0, 1, 101)

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.plot(xi, l1[0](xi), label=r'$\ell_{0}(\xi)$')
ax.plot(xi, l1[1](xi), label=r'$\ell_{1}(\xi)$')
ax.legend(fontsize=14)
ax.plot([0, 1], [0, 0], 'ko')
ax.plot([0, 1], [1, 1], 'ko', mfc='None')
ax.grid()
ax.set_title('Lagrange basis functions (degree 1)')
ax.set_xlabel(r'$\xi$', size=14)
fig.canvas.draw()

In [ ]:
# Lagrange interpolator on primal grid, of degree 1
I0 = Interpolator_from_C0_to_V0(xp, basis=l1)

# Restriction: from 0-form to C0
f0 = f(xp)

# Interpolation: from C0 to V0
fh = I0(f0)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.set_title('Interpolation on primal grid (degree 1)')
ax.plot(x , f(x) , label=r'$\psi\;\; \in \Lambda_0$')
ax.plot(xp, f0   , 'ko', mfc='None', ms=8, mew=1.5, label=r'$\psi^0 \in C_0$')
ax.plot(x , fh(x), label=r'$\psi_{h} \in V_0$')
ax.legend(fontsize=14)
ax.plot(xp, np.zeros_like(xp), 'k.', ms=10)
ax.grid()
ax.set_xlabel('x')
fig.canvas.draw()

In [ ]:
# Convergence test
N_list = [10, 20, 40, 80, 160]
err_list = []

for Ni in N_list:
    dx_i, xp_i, xd_i = staggered_grid(xmin, xmax, Ni)
    I0_i = Interpolator_from_C0_to_V0(xp_i, basis=l1)
    f0_i = f(xp_i)
    fh_i = I0_i(f0_i)
    err_list.append(max(abs(f(x) - fh_i(x))))

print_error_analysis(N_list, err_list)

### Degree 3

Here we use the points $\left(\xi_j\right)_j = \{-1, 0, 1, 2\}$ to define the Lagrange basis functions

\begin{alignat}{2}
\ell^3_{-1}(\xi) &= \frac{\xi~(\xi - 1)~(\xi - 2)}{(-1-0)~(-1-1)~(-1-2)}
                &&= - \frac{\xi^{3}}{6} + \frac{\xi^{2}}{2} - \frac{\xi}{3} \\
\ell^3_0(\xi) &= \frac{(\xi + 1)~(\xi - 1)~(\xi - 2)}{(0+1)~(0-1)~(0-2)}
             &&= \phantom{-}\frac{\xi^{3}}{2} - \xi^{2} - \frac{\xi}{2} + 1 \\
\ell^3_1(\xi) &= \frac{(\xi + 1)~(\xi - 0)~(\xi - 2)}{(1+1)~(1-0)~(1-2)}
             &&= - \frac{\xi^{3}}{2} + \frac{\xi^{2}}{2} + \xi \\
\ell^3_2(\xi) &= \frac{(\xi + 1)~(\xi - 0)~(\xi - 1)}{(2+1)~(2-0)~(2-1)}
             &&= \phantom{-}\frac{\xi^{3}}{6} - \frac{\xi}{6} \\
\end{alignat}

It is easy to verify that $\sum_j \ell^3_j \equiv 1$.

In [ ]:
l3 = Basis_V0(degree=3)

In [ ]:
xi = np.linspace(-1, 2, 101)

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.plot(xi, l3[-1](xi), label=r'$\ell_{-1}(\xi)$')
ax.plot(xi, l3[ 0](xi), label=r'$\ell_{0}(\xi)$')
ax.plot(xi, l3[ 1](xi), label=r'$\ell_{1}(\xi)$')
ax.plot(xi, l3[ 2](xi), label=r'$\ell_{2}(\xi)$')
ax.legend(fontsize=14)
ax.plot([-1, 0, 1, 2], [0, 0, 0, 0], 'ko')
ax.plot([-1, 0, 1, 2], [1, 1, 1, 1], 'ko', mfc='None')
ax.grid()
ax.set_title('Lagrange basis functions (degree 3)')
ax.set_xlabel(r'$\xi$', size=14)
fig.canvas.draw()

In [ ]:
# Lagrange interpolation operator on primal grid, of degree 3
I0 = Interpolator_from_C0_to_V0(xp, basis=l3)

# Restriction: from 0-form to C0
f0 = f(xp)

# Interpolation: from C0 to V0
fh = I0(f0)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.set_title('Interpolation on primal grid (degree 3)')
ax.plot(x , f(x) , label=r'$\psi\;\; \in \Lambda_0$')
ax.plot(xp, f0   , 'ko', mfc='None', ms=8, mew=1.5, label=r'$\psi^0 \in C_0$')
ax.plot(x , fh(x), label=r'$\psi_{h} \in V_0$')
ax.legend(fontsize=14)
ax.plot(xp, np.zeros_like(xp), 'k.', ms=10)
ax.grid()
ax.set_xlabel('x')
fig.canvas.draw()

In [ ]:
# Convergence test
N_list = [10, 20, 40, 80, 160]
err_list = []

for Ni in N_list:
    dx_i, xp_i, xd_i = staggered_grid(xmin, xmax, Ni)
    I0_i = Interpolator_from_C0_to_V0(xp_i, basis=l3)
    f0_i = f(xp_i)
    fh_i = I0_i(f0_i)
    err_list.append(max(abs(f(x) - fh_i(x))))

print_error_analysis(N_list, err_list)

## Interpolation of point values on the dual grid ($\tilde{C}_0 \to \tilde{V}_{0}$)

This is similar to interpolation on the primal grid. We use the same canonical Lagrange basis on a uniform grid in the normalized coordinate $\tilde{\xi} := \frac{x - \tilde{x}_i}{\Delta x}$, where $\tilde{x}_i$ is the coordinate of the $i$-th point on the dual grid.

The interpolation operator is

$$
\tilde{\mathcal{I}}_0: \tilde{\psi}^0 \mapsto \tilde{\psi}_h,
\qquad
\text{with}
\quad
\tilde{\psi}_h: x \mapsto P^d_i\left(\frac{x-\tilde{x}_i}{\Delta x}\right)
\qquad
\text{and}
\quad
i = \left\lfloor {\frac{x - \tilde{x}_0}{\Delta x}} \right\rfloor.
$$

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.plot(xd, np.zeros_like(xp), 'k.', ms=10, label='Dual grid')
ax.plot(x , f(x), label=r'$\psi\;\; \in \Lambda_0$')
ax.plot(xd, f(xd), 'ko', mfc='None', ms=8, mew=1.5, label=r'$\tilde{\psi}^0 \in \tilde{C}_0$')
ax.legend(fontsize=14)

for xdi in xd:
    ax.plot([xdi, xdi], [0, f(xdi)], 'k--')

ax.grid()
ax.set_title(r'Reduction on dual grid, $\tilde{\mathcal{R}}_0: \Lambda_0 \to \tilde{C}_0$')
ax.set_xlabel('x', size=14)
fig.canvas.draw()

### Degree 1

In [ ]:
# Lagrange interpolator on primal grid, using basis of degree 1
I0_t = Interpolator_from_C0_to_V0(xd, basis=l1)

# Restriction: from 0-form to C0
f0_t = f(xd)

# Interpolation: from C0 to V0
fh_t = I0_t(f0_t)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.set_title('Interpolation on dual grid (degree 1)')
ax.plot(x , f(x), label=r'$\psi\;\; \in \Lambda_0$')
ax.plot(xd, f0_t, 'ko', mfc='None', ms=8, mew=1.5, label=r'$\tilde{\psi}^0 \in \tilde{C}_0$')
ax.plot(x , fh_t(x), label=r'$\tilde{\psi}_{h} \in \tilde{V}_0$')
ax.legend(fontsize=14)
ax.plot(xd, np.zeros_like(xd), 'k.', ms=10)
ax.grid()
ax.set_xlabel('x')
fig.canvas.draw()

In [ ]:
# Convergence test
N_list = [10, 20, 40, 80, 160]
err_list = []

for Ni in N_list:
    dx_i, xp_i, xd_i = staggered_grid(xmin, xmax, Ni)
    I0t_i = Interpolator_from_C0_to_V0(xd_i, basis=l1)
    f0t_i = f(xd_i)
    fht_i = I0t_i(f0t_i)
    err_list.append(max(abs(f(x) - fht_i(x))))

print_error_analysis(N_list, err_list)

### Degree 3

In [ ]:
# Lagrange interpolator on primal grid, using basis of degree 1
I0_t = Interpolator_from_C0_to_V0(xd, basis=l3)

# Restriction: from 0-form to C0
f0_t = f(xd)

# Interpolation: from C0 to V0
fh_t = I0_t(f0_t)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.set_title('Interpolation on dual grid (degree 3)')
ax.plot(x , f(x), label=r'$\psi\;\; \in \Lambda_0$')
ax.plot(xd, f0_t, 'ko', mfc='None', ms=8, mew=1.5, label=r'$\tilde{\psi}^0 \in \tilde{C}_0$')
ax.plot(x , fh_t(x), label=r'$\tilde{\psi}_{h} \in \tilde{V}_0$')
ax.legend(fontsize=14)
ax.plot(xd, np.zeros_like(xd), 'k.', ms=10)
ax.grid()
ax.set_xlabel('x')
fig.canvas.draw()

In [ ]:
# Convergence test
N_list = [10, 20, 40, 80, 160]
err_list = []

for Ni in N_list:
    dx_i, xp_i, xd_i = staggered_grid(xmin, xmax, Ni)
    I0t_i = Interpolator_from_C0_to_V0(xd_i, basis=l3)
    f0t_i = f(xd_i)
    fht_i = I0t_i(f0t_i)
    err_list.append(max(abs(f(x) - fht_i(x))))

print_error_analysis(N_list, err_list)

## Interpolation of cell integrals on the primal grid ($C_{1} \to V_{1}$)

Given the interpolation operator $\mathcal{I}_0: C_0 \to V_0$, we want to find the interpolation operator $\mathcal{I}_1: C_{1} \to V_{1}$ that reconstructs a piecewise polynomial $\varphi_h$ from its cell integrals $\varphi^1$.
We assume that $V_1$ is the image of $V_0$ through the derivative operator $\frac{d}{dx}$. Therefore, since $V_0$ is a space of continuous piecewise polynomials of degree $d=2p+1$, $V_1$ is a space of discontinuous piecewise polynomials of degree $d-1=2p$. On each open interval $\left]x_i, x_{i+1}\right[$, the reconstructed function is

$$
\varphi_h(x) = \sum_{j=-p}^{p} \varphi^1_j e_j(x),
$$

where the functions $(e_j)_j$ constitute a canonical basis of $V_1$, in the sense that $\int_{x_{i+k}}^{x_{i+k+1}} e_j(x)~dx = \delta_{jk}$.
The interpolation operator $\mathcal{I}_1$ is completely defined if we can write the canonical basis of $V_1$ in terms of the canonical basis of $V_0$: in order to achive this, we ask $\mathcal{I}_1$ to satisfy the commuting diagram property

$$
\frac{d}{dx}\left(\mathcal{I}_0 \psi^0\right) = \mathcal{I}_1 \mathbb{G}~\psi^0,
\qquad
\forall~\psi^0 \in C_0.
$$



Given that $\psi_h \in V_0$ is a Lipschitz continuous functions that is usually _not differentiable_ at the grid points $x_i$, the above statement holds in a _weak_ sense, and it holds in a strong sense on each open interval $\left]x_i, x_{i+1}\right[$.

If we focus on a single cell and omit its index $i$ for simplicity, the left-hand side of the equation above reads

$$
\frac{d}{dx} \sum_{j=-p}^{p+1}~\psi^0_j \ell'_j
= \sum_{j=-p}^{p+1}~\psi^0_j \ell_j'
= \psi^0_{-p} \ell_{-p}' + \sum_{j=-p+1}^{p}~\psi^0_j \ell_j' + \psi^0_{p+1} \ell_{p+1}',
$$

while the right-hand side reads

$$
\sum_{j=-p}^{p}~\left(\mathbb{G}~\psi^0\right)_j e_j
= \sum_{j=-p}^{p}~\left(\psi^0_{j+1}-\psi^0_j\right) e_j
= \psi^0_{-p} \left(-e_{-p}\right) + \sum_{j=-p+1}^p \psi^0_j \left(e_{j-1} - e_j\right) + \psi^0_{p+1} e_p.
$$

Equating the two expressions term-by-term yields a linear system of $2p+1$ equations in $2p$ unknowns,

$$
\begin{bmatrix}
-1     &        & \dots  &  0     \\
 1     & -1     &        & \vdots \\
       & \ddots & \ddots &        \\
\vdots &        &  1     & -1     \\
 0     & \dots  &        &  1
\end{bmatrix}
\begin{array}{@{}c@{}}
\begin{bmatrix}
e_{-p} \\
\vdots \\
\\
e_{p}
\end{bmatrix}\\
\vphantom{\vdots}\\
\end{array}
=
\begin{bmatrix}
\ell'_{-p}\\
\vdots \\
\\
\ell'_{p}\\
\ell'_{p+1}
\end{bmatrix},
$$

where one equation is redundant: the side-by-side sum of all equations yields
$$
0 = \sum_{j=-p}^{p+1} \ell'_j,
$$
an identity that follows from the partition of unity property of the Lagrange basis, as
$$
\frac{d}{dx} \sum_{j=-p}^{p+1} \ell_j(x) = \frac{d}{dx}(1) = 0
\qquad
\forall x.
$$
In order to solve for the unknowns $\left(e_j\right)_j$ it is sufficient to arbitrarily remove one equation from the linear system above.
If we invert the matrix after removing the first equation, we obtain an upper triangular matrix full of ones:

$$
\begin{bmatrix}
e_{-p} \\
\\
\vdots \\
\\
e_{p}
\end{bmatrix}
=
\begin{bmatrix}
1      & 1     &        & \dots  &  1     \\
       & 1     & 1      &        & \vdots \\
       &       & \ddots & \ddots &        \\
\vdots &       &        & 1      & 1      \\
 0     & \dots &        &        & 1
\end{bmatrix}
\begin{bmatrix}
\ell'_{-p+1}\\
\\
\vdots \\
\\
\ell'_{p+1}
\end{bmatrix}.
$$

Instead, if we invert the matrix after removing the last equation we obtain a lower triangular matrix full of $(-1)$ values:

$$
\begin{bmatrix}
e_{-p} \\
\\
\vdots \\
\\
e_{p}
\end{bmatrix}
=
\begin{bmatrix}
-1     &        &        & \dots &  0     \\
-1     & -1     &        &       & \vdots \\
       & \ddots & \ddots &       &        \\
\vdots &        & -1     & -1    &        \\
-1     & \dots  &        & -1    & -1
\end{bmatrix}
\begin{bmatrix}
\ell'_{-p}\\
\\
\vdots \\
\\
\ell'_{p}
\end{bmatrix}.
$$

Finally, we may also add the missing column to each of the two matrices above and take their average:

$$
\begin{bmatrix}
e_{-p} \\
\\
\vdots \\
\\
e_{p}
\end{bmatrix}
=
\frac{1}{2}~
\begin{bmatrix}
-1     &  1    &        &        & \dots & 1      \\
       & -1    & 1      &        &       & \vdots \\
       &       & \ddots & \ddots &       &        \\
\vdots &       &        & -1     &  1    &        \\
-1     & \dots &        &        & -1    & 1
\end{bmatrix}
\begin{array}{@{}c@{}}
\vphantom{\vdots}\\
\begin{bmatrix}
\ell'_{-p}\\
\\
\vdots \\
\\
\ell'_{p}\\
\ell'_{p+1}\\
\end{bmatrix}.
\end{array}
$$

This yields the symmetric formula
$$
e_i = -~\frac{1}{2} \sum_{j=-p}^{i} \ell'_j + \frac{1}{2}\sum_{j=i+1}^{p+1} \ell'_j
\qquad
\text{or}
\qquad
e_i = \frac{1}{2} \sum_{j=-p}^{p+1} \text{sgn}\left(j-i-\frac{1}{2}\right)~\ell'_j.
$$

In [ ]:
# Integral of function g over interval [a, b]
# Usage: I(a, b)(g)
integral = lambda a, b: (lambda g: quad(g, a, b)[0])

### Degree 1

From the Lagrange basis functions of degree 1 we derive only one histopolation basis function, which is constant:

$$
e^1_0(\xi) = 1.
$$

The partition of unity property, $\sum_j e^1_j(\xi) \equiv 1$, is trivially satisfied in this case.

In [ ]:
e1 = Basis_V1(degree=1)

In [ ]:
xi = np.linspace(0, 1, 101)

fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.plot(xi, e1[0](xi), label=r'$e_{0}(\xi)$')
ax.legend(fontsize=14)
ax.plot([0, 1], [0, 0], 'ko')
ax.plot([0, 1], [1, 1], 'ko', mfc='None')
ax.grid()
ax.set_title('Histopolation basis functions (degree 1)')
ax.set_xlabel(r'$\xi$')
fig.canvas.draw()

In [ ]:
# Lagrange histopolation operator on primal grid, of degree 1
I1 = Interpolator_from_C1_to_V1(xp, basis=e1)

# Restriction: from 1-form to C1
phi1 = [integral(xj, xj+dx)(f) / dx for xj in xp]

# Reconstruction: from C1 to V1
phi_h = I1(phi1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.plot(x, f(x)    , label=r'$\phi\in\Lambda_1$')
ax.plot(x, phi_h(x), label=r'$\phi_{h}\in V_1$')
ax.legend(fontsize=14)
ax.grid()
ax.set_title('Histopolation on primal grid (degree 1)')
ax.set_xlabel('x')
fig.canvas.draw()

In [ ]:
# Convergence test
N_list = [10, 20, 40, 80, 160]
err_list = []

for Ni in N_list:
    dx_i, xp_i, xd_i = staggered_grid(xmin, xmax, Ni)
    I1_i = Interpolator_from_C1_to_V1(xp_i, basis=e1)
    F1_i = [integral(xj, xj+dx_i)(f) / dx_i for xj in xp_i]
    Fh_i = I1_i(F1_i)
    err_list.append(max(abs(f(x) - Fh_i(x))))

print_error_analysis(N_list, err_list)

### Degree 3

From the Lagrange basis functions of degree 3 we derive three histopolation basis functions, namely

$$
\begin{aligned}
e^3_{-1}(\xi) &= \frac{1}{2}\xi^2 - \xi + \frac{1}{3}\\
e^3_{ 0}(\xi) &= -\xi^2           + \xi + \frac{5}{6}\\
e^3_{ 1}(\xi) &= \frac{1}{2}\xi^2       - \frac{1}{6}
\end{aligned}
$$

It is easy to verify that $\sum_j e^3_j(\xi) \equiv 1$.

In [ ]:
e3 = Basis_V1(degree=3)

In [ ]:
xi = np.linspace(-1, 2, 101)

fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.plot(xi, e3[-1](xi), label=r'$e_{-1}(x)$')
ax.plot(xi, e3[ 0](xi), label=r'$e_{0}(x)$')
ax.plot(xi, e3[ 1](xi), label=r'$e_{1}(x)$')
ax.legend(fontsize=14)
ax.plot([-1, 0, 1, 2], [0, 0, 0, 0], 'ko')
ax.plot([-1, 0], [0, 0], 'k')
ax.plot([ 0, 1], [1, 1], 'k')
ax.plot([ 1, 2], [0, 0], 'k')
ax.grid()
ax.set_title('Histopolation basis functions on primal grid (degree 3)')
ax.set_xlabel('x')
fig.canvas.draw()

In [ ]:
# Lagrange histopolation operator on primal grid, of degree 3
I1 = Interpolator_from_C1_to_V1(xp, basis=e3)

# Restriction: from 1-form to C1
phi1 = [integral(xj, xj+dx)(f) / dx for xj in xp]

# Reconstruction: from C1 to V1
phi_h = I1(phi1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.plot(x, f(x)    , label=r'$\phi \in \Lambda_1$')
ax.plot(x, phi_h(x), label=r'$\phi_{h} \in V_1$')
ax.legend(fontsize=14)
ax.grid()
ax.set_title('Histopolation on primal grid (degree 3)')
ax.set_xlabel('x')
fig.canvas.draw()

In [ ]:
# Convergence test
N_list = [10, 20, 40, 80, 160]
err_list = []

for Ni in N_list:
    dx_i, xp_i, xd_i = staggered_grid(xmin, xmax, Ni)
    I1_i = Interpolator_from_C1_to_V1(xp_i, basis=e3)
    F1_i = [integral(xj, xj+dx_i)(f) / dx_i for xj in xp_i]
    Fh_i = I1_i(F1_i)
    err_list.append(max(abs(f(x) - Fh_i(x))))

print_error_analysis(N_list, err_list)

# Hodge going from 1-form to 0-form

Let's look at the Hodge operator that maps integral values on the dual grid onto point values on the primal grid

$$
\mathbb{H}_0 :  \tilde{\mathbb{C}}_1 \to \mathbb{C}_0,
\quad
\tilde{\varphi}^1 \mapsto \mathbb{H}_0 \left(\tilde{\varphi}^1\right) = \psi^0.
$$

We can choose it to satisfy the commuting diagram property

$$
\mathbb{H}_0 := \mathcal{R}_0 \star \tilde{\mathcal{I}}_1,
$$

where

$$
\begin{alignedat}{6}
\tilde{\mathcal{I}}_1 &: \tilde{\mathbb{C}}_1 \to V_1 \subset \Lambda_1
     \qquad && \tilde{\varphi}^1 &\mapsto&~\tilde{\mathcal{I}}_1\left(\tilde{\varphi}^1\right) = \varphi_h
     \qquad && \text{histopolation on dual grid}
     \qquad\qquad && \varphi_h(x) = \sum_{k=-p}^p \tilde{\varphi}^1_{i+k}~\tilde{e}_k(x)\\
\star &: \Lambda_1 \to \Lambda_0
     && \star &\mapsto& \star (\varphi) = \psi
     && \text{identity}
     && \psi(x) = \star \varphi_h(x) = \sum_{k=-p}^p \tilde{\varphi}^1_{i+k}~\tilde{e}_k(x) \\
\mathcal{R}_0 &: \Lambda_0 \to \mathbb{C}_0
     && \psi &\mapsto&~\mathcal{R}_0(\psi) = \psi^0
     && \text{evaluation on primal grid}
     && \psi^0_i = \mathcal{R}_0(\psi)_i = \psi \left(x_i\right) = \sum_{k=-p}^p \tilde{\varphi}^1_{i+k}~\tilde{e}_k(x_i)
\end{alignedat}
$$

We know that $\psi^0 = \mathbb{H}_0 \tilde{\varphi}^1$, hence we can write the entries of the Hodge matrix as

$$
\left(\mathbb{H}_0\right)_{ij} =
\begin{cases}
  \frac{1}{\Delta x} ~ e_{i-j}\left(\frac{1}{2}\right) & \text{if $j \in [i-p, i+p]$},\\
  0 & \text{otherwise}.
\end{cases}
$$

where we have used the "canonical" basis functions in the normalized variable $\xi$.

Note that if we choose to define $\mathbb{H}_0$ according to this formula, in the code we must make the approximation $\tilde{\mathbb{H}}_1 \approx \left(\mathbb{H}_0\right)^{-1}$.

In [ ]:
# Construct the Hodge matrix that maps (discrete) 1-forms to (discrete) 0-forms

# NOTE: result should be divided by dx!
def Hodge0(N, degree):
    e = Basis_V1(degree)
    p = degree // 2
    c = np.zeros(N)

    c[:p+1] = [e[i](0.5) for i in range(p+1)]
    
    if p > 0:
        c[-p: ] = [e[i](0.5) for i in range(-p, 0)]

    return circulant(c)

np.set_printoptions(linewidth=130, precision=5)

### Degree 1

In [ ]:
H0_d1 = Hodge0(N, 1)
H0_d1

### Degree 3

In [ ]:
H0_d3 = Hodge0(N, 3)
H0_d3

In [ ]:
print('Check:')
print('------')
print('26/24 = {}'.format(26/24))
print('-1/24 = {}'.format(-1/24))

### Degree 5

In [ ]:
H0_d5 = Hodge0(N, 5)
H0_d5

In [ ]:
print('Check:')
print('------')
print('1067/960 = {}'.format(1067/960))
print(' -29/480 = {}'.format( -29/480))
print('   3/640 = {}'.format(   3/640))

# Hodge going from 0-form to 1-form

Let's look at the Hodge operator that maps point values on the primal grid onto integral values on the dual grid

$$
\tilde{\mathbb{H}}_1 :  \mathbb{C}_0 \to \tilde{\mathbb{C}}_1,
\quad
\psi^0 \mapsto \tilde{\mathbb{H}}_1 \left(\psi^0\right) = \tilde{\varphi}^1.
$$

We can choose it to satisfy the commuting diagram property

$$
\tilde{\mathbb{H}}_1 := \tilde{\mathcal{R}}_1 \star \mathcal{I}_0,
$$

where

$$
\begin{alignedat}{6}
\mathcal{I}_0 &: \mathbb{C}_0 \to V_0 \subset \Lambda_0
     \qquad && \psi^0 &\mapsto&~\mathcal{I}_0\left(\psi^0\right) = \psi_h
     \qquad && \text{interpolation on primal grid}
     \qquad\qquad && \psi_h(x) = \sum_{k=-p}^{p+1} \psi^0_{i+k}~\ell_k(x)\\
\star &: \Lambda_0 \to \Lambda_1
     && \star &\mapsto& \star (\psi) = \varphi
     && \text{identity}
     && \varphi(x) = \star \psi_h(x) = \sum_{k=-p}^{p+1} \psi^0_{i+k}~\ell_k(x) \\
\tilde{\mathcal{R}}_1 &: \Lambda_1 \to \tilde{\mathbb{C}}_1
     && \varphi &\mapsto&~\tilde{\mathcal{R}}_1(\varphi) = \tilde{\varphi}^1
     && \text{integration on dual grid}
     && \tilde{\varphi}^1_i = \tilde{\mathcal{R}}_1(\varphi)_i = \int_{\tilde{x}_{i-1}}^{\tilde{x}_i} \varphi(x)~dx = \dots
\end{alignedat}
$$

Let's take a closer look at the last integral:

$$
\begin{aligned}
\tilde{\varphi}^1_i &= \tilde{\mathcal{R}}_1(\varphi)_i = \int_{\tilde{x}_{i-1}}^{\tilde{x}_i} \varphi(x)~dx
    = \int_{x_i - \frac{\Delta x}{2}}^{x_i} \varphi(x)~dx + \int_{x_i}^{x_i + \frac{\Delta x}{2}} \varphi(x)~dx \\
&= \sum_{k=-p-1}^{p} \psi^0_{i+k} \left[ \Delta x \int_{\frac{1}{2}}^{1} \ell_{k+1}(\xi)~d\xi \right] +
   \sum_{k=-p}^{p+1} \psi^0_{i+k} \left[ \Delta x \int_{0}^{\frac{1}{2}} \ell_{k  }(\xi)~d\xi \right]
\end{aligned}
$$

From this we get that $\tilde{\varphi}^1 = A \psi^0 + B \psi^0$ with

$$
A_{ij} =
\begin{cases}
\Delta x \int_{\frac{1}{2}}^{1}{\ell_{j-i+1}(\xi)~d\xi} & \text{if $j \in [i-p-1, i+p]$},\\
0 & \text{otherwise},
\end{cases}
\qquad
B_{ij} =
\begin{cases}
\Delta x \int_{0}^{\frac{1}{2}}{\ell_{j-i}(\xi)~d\xi} & \text{if $j \in [i-p, i+p+1]$},\\
0 & \text{otherwise},
\end{cases}
$$

and since we know that $\tilde{\varphi}^1 = \tilde{\mathbb{H}}_1 \psi^0$, we can write the Hodge matrix as $\tilde{\mathbb{H}}_1 = A + B$.

In [ ]:
# Construct the Hodge matrix that maps (discrete) 0-forms to (discrete) 1-forms

# NOTE: result should be multiplied by dx!
def Hodge1(N, degree):
    l = Basis_V0(degree)
    p = degree // 2

    a = np.zeros(N)
    for k in range(-p-1, p+1):
        L    = l[k+1].integ()
        i    = k % N
        a[i] = L(1) - L(0.5)

    b = np.zeros(N)
    for k in range(-p, p+2):
        L    = l[k].integ()
        i    = k % N
        b[i] = L(0.5) - L(0)

    return circulant(a + b)

### Degree 1

In [ ]:
H1_d1 = Hodge1(N, 1)
H1_d1

In [ ]:
print('Check:')
print('------')
print('6/8 = {}'.format(6/8))
print('1/8 = {}'.format(1/8))

### Degree 3

In [ ]:
H1_d3 = Hodge1(N, 3)
H1_d3

In [ ]:
print('Check:')
print('------')
print('310/384 = {}'.format(310/384))
print(' 44/384 = {}'.format( 44/384))
print(' -7/384 = {}'.format( -7/384))

### Degree 5

In [ ]:
H1_d5 = Hodge1(N, 5)
H1_d5